In [1]:
import yaml
from yaml.loader import SafeLoader
import numpy as np

In [2]:
# Open the file and load the file
with open('IEA-15-240-RWT_test.yaml') as f:
    data = yaml.load(f, Loader=SafeLoader)

In [3]:
print(data.keys())

dict_keys(['name', 'description', 'assembly', 'simulationparamter', 'environment', 'components', 'airfoils', 'materials', 'control', 'bos', 'costs'])


# Constant Parameters

In [4]:
scaling_blade = 1; 
scaling_tower = 1;
cos_msl       = [0.0, 0.0, 0.0]                                                 # MSL position in global coordinate system

n_yaw         = [0, 0, 1]                                                 # yaw rotation axis
n_tilt        = [0, 1, 0]                                                # tilt rotation axis

flag_tower       = 1;                                                  # - tower on(1)/off(0)
flag_tower_aero  = 0;                                                  # - tower aero grid on(1)/off(0)
flag_tower_struc = 1;                                                  # - tower aero grid on(1)/off(0)

flag_blade       = 1;                                                  # - blades on(1)/off(0)
flag_blade_aero  = 1;                                                  # - blades aero grid on(1)/off(0)
flag_blade_struc = 1;                                                  # - blades structure mesh on(1)/off(0)

flag_foundation       = 0;                                             # - foundation on(1)/off(0)
flag_foundation_aero  = 0;                                             # - foundation aero grid on(1)/off(0)
flag_foundation_struc = 0;                                             # - foundation structure mesh on(1)/off(0)

flag_hub     = 1;                                                      # - hub on(1)/off(0)
flag_nacelle = 1;  

In [5]:
mspan = data["components"]["blade"]["DeSiO"]["uvlm"]["M_aero"]
print(f"mspan : {mspan}")

mspan : 50


In [6]:
jobname      = 'mesh_tests_blade'; 
if "jobname" in data.keys():
    strfilename = data["jobname"]
strfilename = jobname +'_pitch' + str(data["environment"]["pitch_angle"])+ '_vel' +str(data["environment"]["vinf"])

# Environmnet Conditions

In [7]:
yaw_angle = 0.0;
pitch_angle = 0.0;
if "environment" in data.keys():
    if "yaw_angle" in data["environment"].keys():
        yaw_angle = data["environment"]["yaw_angle"]
    if "pitch_angle" in data["environment"].keys():
        pitch_angle = data["environment"]["pitch_angle"]
fsi_radius_rbf = 0;
if 'fsi_radius_rbf' in data["simulationparamter"].keys():
    fsi_radius_rbf = data["simulationparamter"]["fsi_radius_rbf"];

In [8]:
from scipy.interpolate import interp1d
from math import pi, cos, sin

class Beam:
    def __init__(self,strName,model_beam, materials):
        
        self.strName = strName
        self.materials = materials
        # span wise discritization
        self.model_beam = model_beam
        self.M = model_beam["M_struc"]
        # initializing stiffness and mass matrix
        self.arr_stiff_matrix = np.zeros((self.M,21))
        self.arr_mass_matrix = np.zeros((self.M,6))
        O = np.zeros((self.M,1));
        
        # nodal natural coordinates in span-wise direction
        self.xhi_x  = np.arange(0,1 + 1/self.M,1/self.M)
        
        # element natural coordinates in span-wise direction
        self.xhi_x_elem = (self.xhi_x[0:-1]+self.xhi_x[1:])/2
        self.interpolate_ref_values()
        
        self.connectivity = np.zeros((self.M,3)) 
    def ifassign(self,field, subkey, xq):
        """
            This function checks if the subkey is available in the field.keys
            if it does the it does linear interpolation between grid and values
            Inputs:
                field: dict
                subkey: str
                xq: querry points
        """
        if subkey in field.keys():
            x = field[subkey]["grid"]
            y = field[subkey]["values"]
            fun = interp1d(x,y)
            return fun(xq)
    def interpolate_cross_sectional(self):
        # if-statement according to type of surface cross-section
        if "blade" in self.strName:
            # interpolating along span-wise direction
            if 'six_x_six' in self.model_beam["elastic_properties_mb"]:
                x_el_phy = self.model_beam["elastic_properties_mb"]["six_x_six"]["stiff_matrix"]["grid"]
                
                self.model_beam["elastic_properties_mb"]["six_x_six"]["stiff_matrix"]["values"] = np.asarray(self.model_beam["elastic_properties_mb"]["six_x_six"]["stiff_matrix"]["values"])
                for i in range(21):
                    
                    
                    x_el_ref = self.model_beam["elastic_properties_mb"]["six_x_six"]["stiff_matrix"]["values"][:,i]
                    
                    el_model = interp1d(x_el_phy, x_el_ref)
                    self.arr_stiff_matrix[:,i] = el_model(self.xhi_x_elem)
                x_in_phy = self.model_beam["elastic_properties_mb"]["six_x_six"]["inertia_matrix"]["grid"]
                self.model_beam["elastic_properties_mb"]["six_x_six"]["inertia_matrix"]["values"] = np.asarray(self.model_beam["elastic_properties_mb"]["six_x_six"]["inertia_matrix"]["values"])
                for i in range(6):
                    x_in_ref = self.model_beam["elastic_properties_mb"]["six_x_six"]["inertia_matrix"]["values"][:,i]
                    in_model = interp1d(x_in_phy, x_in_ref)
                    self.arr_mass_matrix[:,i] = in_model(self.xhi_x_elem)
                self.dissipation = np.asarray([self.model_beam["dissipation"]["alpha_s"], self.model_beam["dissipation"]["alpha_v"]])
            else:
                self.arr_EA  = np.zeros((self.M,1)); 
                self.arr_GA1 = np.zeros((self.M,1)); 
                self.arr_GA2 = np.zeros((self.M,1));
                self.arr_EI1 = np.zeros((self.M,1)); 
                self.arr_EI2 = np.zeros((self.M,1)); 
                self.arr_GI3 = np.zeros((self.M,1));
                self.arr_ES1 = np.zeros((self.M,1)); 
                self.arr_ES2 = np.zeros((self.M,1)); 
                self.arr_EI12 = np.zeros((self.M,1));
                self.arr_GS1 = np.zeros((self.M,1)); 
                self.arr_GS2 = np.zeros((self.M,1));

                self.arr_rhoA  = np.zeros((self.M,1)); self.arr_rhoI1 = np.zeros((self.M,1)); self.arr_rhoI2  = np.zeros((self.M,1));
                self.arr_rhoS1 = np.zeros((self.M,1)); self.arr_rhoS2 = np.zeros((self.M,1)); self.arr_rhoI12 = np.zeros((self.M,1));

                if 'EA' in self.model_beam["elastic_properties_mb"].keys():
                    x = self.model_beam["elastic_properties_mb"]["EA"]["grid"]
                    y = self.model_beam["elastic_properties_mb"]["EA"]["values"]
                    fun = interp1d(x,y)
                    self.arr_EA   = fun(xhi_x_elem);


                
                self.arr_GA1  = self.ifassign(self.model_beam["elastic_properties_mb"],'GA1',self.xhi_x_elem)
                self.arr_GA2  = self.ifassign(self.model_beam["elastic_properties_mb"],'GA2',self.xhi_x_elem)
                self.arr_EI1  = self.ifassign(self.model_beam["elastic_properties_mb"],'EI1',self.xhi_x_elem)
                self.arr_EI2  = self.ifassign(self.model_beam["elastic_properties_mb"],'EI2',self.xhi_x_elem)
                self.arr_GI3  = self.ifassign(self.model_beam["elastic_properties_mb"],'GI3',self.xhi_x_elem)
                self.arr_ES1  = self.ifassign(self.model_beam["elastic_properties_mb"],'ES1',self.xhi_x_elem)
                self.arr_ES2  = self.ifassign(self.model_beam["elastic_properties_mb"],'ES2',self.xhi_x_elem)
                self.arr_GS1  = self.ifassign(self.model_beam["elastic_properties_mb"],'GS1',self.xhi_x_elem)
                self.arr_GS2  = self.ifassign(self.model_beam["elastic_properties_mb"],'GS2',self.xhi_x_elem)
                self.arr_EI12  = self.ifassign(self.model_beam["elastic_properties_mb"],'EI12',self.xhi_x_elem)
                self.arr_rhoA  = self.ifassign(self.model_beam["elastic_properties_mb"],'rhoA',self.xhi_x_elem)
                self.arr_rhoI1  = self.ifassign(self.model_beam["elastic_properties_mb"],'rhoI1',self.xhi_x_elem)
                self.arr_rhoI2  = self.ifassign(self.model_beam["elastic_properties_mb"],'rhoI2',self.xhi_x_elem)
                self.arr_rhoS1  = self.ifassign(self.model_beam["elastic_properties_mb"],'rhoS1',self.xhi_x_elem)
                self.arr_rhoS2  = self.ifassign(self.model_beam["elastic_properties_mb"],'rhoS2',self.xhi_x_elem)
                self.arr_rhoI12  = self.ifassign(self.model_beam["elastic_properties_mb"],'rhoI12',self.xhi_x_elem)

                
                # GA1 GA2 EA EI1 EI2 GI3 0 0 0 GS2 -GS1 0 0 0 0 0 ES1 -EI12 -ES2 0 0
                self.arr_stiff_matrix[:,:] = [self.arr_GA1, self.arr_GA2, self.arr_EA, self.arr_EI1, self.arr_EI2, self.arr_GI3, \
                                                O, O, O, self.arr_GS2, -self.arr_GS1, O, O, O, O, O, self.arr_ES1, -self.arr_EI12,\
                                              -self.arr_ES2, O, O];

                # rhoA, rhoI2, rhoI1, rhoI12, rhoS1, rhoS2
                self.arr_mass_matrix[:,:]   = [self.arr_rhoA, self.arr_rhoI2, self.arr_rhoI1, self.arr_rhoI12, self.arr_rhoS1, beam.arr_rhoS2];
                self.dissipation         = np.asarry([self.model_beam["dissipation"]["alpha_s"], self.model_beam["dissipation"]["alpha_v"]])
        
        if 'pipe' in self.strName:
            # interpolating cross-section properties along span-wise direction
            E = 0.0; G = 0.0; rho = 0.0;
            if 'material' in self.model_beam.keys():
                strmaterial = self.model_beam["material"];
                for i in range(len( self.materials.keys() )):
                    if self.materials[i]["name"] in strmaterial:
                        if len(self.materials[i]["name"]) == len(strmaterial):
                            E   = self.materials[i].E;
                            nu  = self.materials[i].nu;
                            rho = self.materials[i].rho;
                            G   = E/(2.0*(1.0+nu));




            if 'elastic_properties_mb' in self.model_beam.keys():
                E   = self.model_beam["elastic_properties_mb"]["E"];
                G   = E/(2.0*(1+self.model_beam["elastic_properties_mb"]["nu"]));
                rho = self.model_beam["elastic_properties_mb"]["rho"];


            k1  = 1.0;
            k2  = 1.0;
            if 'shear_factor' in self.model_beam.keys():
                k1 = self.model_beam["shear_factor"]["k1"];
                k2 = self.model_beam["shear_factor"]["k2"];


            self.arr_outer_diameter = self.ifassign(self.model_beam,"outer_diameter",self.xhi_x_elem)
            
            self.arr_thickness      = self.ifassign(self.model_beam,"thickness",self.xhi_x_elem)
            self.arr_EA             = E*pi/4*(self.arr_outer_diameter**2 - (self.arr_outer_diameter-2*self.arr_thickness)**2);
            self.arr_GA1            = k1*G*pi/4*(self.arr_outer_diameter**2 - (self.arr_outer_diameter-2*self.arr_thickness)**2);
            self.arr_GA2            = k2*G*pi/4*(self.arr_outer_diameter**2 - (self.arr_outer_diameter-2*self.arr_thickness)**2);
            self.arr_EI1            = E*pi/64*(self.arr_outer_diameter**4 - (self.arr_outer_diameter-2*self.arr_thickness)**4);
            self.arr_EI2            = E*pi/64*(self.arr_outer_diameter**4 - (self.arr_outer_diameter-2*self.arr_thickness)**4);
            self.arr_GI3            = G*pi/32*(self.arr_outer_diameter**4 - (self.arr_outer_diameter-2*self.arr_thickness)**4);
            self.arr_ES1            = self.arr_EA*0;
            self.arr_ES2            = self.arr_EA*0;
            self.arr_GS2            = self.arr_EA*0;
            self.arr_GS1            = self.arr_EA*0;
            self.arr_EI12           = self.arr_EA*0;

            self.arr_rhoA           = rho*pi/4*(self.arr_outer_diameter**2 - (self.arr_outer_diameter-2*self.arr_thickness)**2);
            self.arr_rhoI1          = rho*pi/64*(self.arr_outer_diameter**4 - (self.arr_outer_diameter-2*self.arr_thickness)**4);
            self.arr_rhoI2          = rho*pi/64*(self.arr_outer_diameter**4 - (self.arr_outer_diameter-2*self.arr_thickness)**4);
            self.arr_rhoI12         = self.arr_rhoA*0;
            self.arr_rhoS1          = self.arr_rhoA*0;
            self.arr_rhoS2          = self.arr_rhoA*0;

            # GA1 GA2 EA EI1 EI2 GI3 0 0 0 GS2 -GS1 0 0 0 0 0 ES1 -EI12 -ES2 0 0
            self.arr_stiff_matrix[:,:] = [self.arr_GA1, self.arr_GA2, self.arr_EA, self.arr_EI1, self.arr_EI2, self.arr_GI3, \
                                            O, O, O, self.arr_GS2, -self.arr_GS1, O, O, O, O, O, self.arr_ES1, -self.arr_EI12, -self.arr_ES2, O, O];

            self.arr_mass_matrix[:,:] = [self.arr_rhoA, self.arr_rhoI2, self.arr_rhoI1, self.arr_rhoI12, self.arr_rhoS1, self.arr_rhoS2];
            self.dissipation       = [ self.model_beam["dissipation"]["alpha_s"], self.model_beam["dissipation"]["alpha_v"]];                                    
                
        
    def interpolate_ref_values(self):
        x_phy = self.model_beam["reference_axis"]["x"]["grid"]
        y_phy = self.model_beam["reference_axis"]["y"]["grid"]
        z_phy = self.model_beam["reference_axis"]["z"]["grid"]
        x_ref = self.model_beam["reference_axis"]["x"]["values"]
        y_ref = self.model_beam["reference_axis"]["y"]["values"]
        z_ref = self.model_beam["reference_axis"]["z"]["values"]
        fx = interp1d(x_phy, x_ref)
        fy = interp1d(y_phy, y_ref)
        fz = interp1d(z_phy, z_ref)
        self.arr_xre_x = fx(self.xhi_x)
        self.arr_xre_y = fx(self.xhi_x)
        self.arr_xre_z = fx(self.xhi_x)
        
        # interpolating twist angle according to span-wise discretization
        if 'twist' in self.model_beam.keys():
            phi_phy = self.model_beam["twist"]["grid"]
            phi_ref = self.model_beam["twist"]["values"]
            phi_interp = interp1d(phi_phy, phi_ref)
            self.arr_twist = phi_interp(self.xhi_x);
        self.interpolate_cross_sectional()
        
        
def fun_extract_beam_data(strName,model_beam,materials):
    """
         input:
           strName - type of surface of (airfoil) cross-section
           model - component beam object specified in WindIO
           materials - material data specified in WindIO
           scale_opt - scaling factor for scaling in longitudinal direction (optional input)
         output:
             beam - beam object containing coordinates and connectivity for creating structural mesh
    """
  
    # span-wise discretization
    beam = Beam(strName,model_beam,materials)
    return beam


In [9]:
def fun_get_beam_model(strName,model,materials,scale_opt=1):
    """
         input:
           strName - type of surface of (airfoil) cross-section
           model - component beam object specified in WindIO
           materials - material data specified in WindIO
           scale_opt - scaling factor for scaling in longitudinal direction (optional input)
         output:
           beam - beam object containing coordinates and connectivity for creating structural mesh
    """
    beam  = fun_extract_beam_data(strName,model,materials)
    scale = scale_opt
    # twist angle around pitch axis
    arr_twist = np.zeros((beam.M+1,1))
    if 'twist' in model.keys():
        arr_twist = -beam.arr_twist;


    # reference axis
    arr_xre   = np.vstack((beam.arr_xre_x,beam.arr_xre_y,beam.arr_xre_z*scale)).T;

    # beam position and director coordinates of the reference axis
    beam.arr_coordinates = [];
    for i in range(model["M_struc"]+1):

        # save the old directors from the previous segment
        if i == 0:
            d1_old = np.asarray([[1], [0], [0]]);
            d2_old = np.asarray([[0], [1], [0]]);
            d3_old = np.asarray([[0], [0], [1]]);
            alpha_old = 0;
        else:

            alpha_old = arr_twist[i-1];


        # compute the new 3-director
        n3 = np.zeros((3, 1));   # the current connection vector
        if i == model["M_struc"]:
            n3[:,0] = arr_xre[i,:] - arr_xre[i-1,:];  # exeption at last node: connection interpolated backwards
        else:
            n3[:,0] = arr_xre[i+1,:] - arr_xre[i,:];  # usual case: connection interpolated forward

        d3 = n3/np.linalg.norm(n3);   # 3-director as normed connection vector

        # compute the angle between old and new 3-director and the
        # corresponding rotation axis
        theta = np.arccos(np.dot(d3_old[:,0], d3[:,0]));   # rotation angle

        # compute the rotation axis for the computation with the new
        # 3-director if the rotation angle is not very small
        if theta <= 1e-6:   # special case (for the last segment): the old and new director are very similar --> then just use one of the old directors as 
            rot_matrix = np.eye(3);
        else:
            temp = np.cross(d3_old[:,0], d3[:,0])
            rot_axis = temp/np.linalg.norm(temp);   # direction vector of the rotation axis
            u_x = rot_axis[0];  # 1-component of the rotation axis
            u_y = rot_axis[1];  # 2-component of the rotation axis
            u_z = rot_axis[2];  # 3-component of the rotation axis

        # compute the rotation matrix
            rot_matrix = np.asarray([
            [cos(theta) + u_x**2*(1-cos(theta)),          u_x*u_y*(1-cos(theta)) - u_z*sin(theta),	u_x*u_z*(1-cos(theta)) + u_y*sin(theta)],
            [u_y*u_x*(1-cos(theta)) + u_z * sin(theta),    cos(theta) + u_y**2*(1-cos(theta)),          u_y*u_z*(1-cos(theta)) - u_x*sin(theta)],
            [u_z*u_x*(1-cos(theta)) - u_y*sin(theta),    u_z*u_y*(1-cos(theta)) + u_x*sin(theta),    cos(theta) + u_z**2*(1-cos(theta))]
                    ]);


        # rotate the directors to get the new COS (without twist)
        d3_check = rot_matrix @ d3_old;
        d3_diff = d3 - d3_check;
        if np.linalg.norm(d3_diff)>1e-10:
            print('ERROR: rotation does not work as inted');
            print(d3_old);
            print(d3);
            print(d3_check);
            return

        d2_noTw = rot_matrix @ d2_old
        d1_noTw = rot_matrix @ d1_old

        # include the twist
        alpha_tw = arr_twist[i] - alpha_old;   # current twist angle

        # rotation axis for the twist rotation: the current 3-director
        rot_axis = d3;
        u_x = float(rot_axis[0])  # 1-component of the rotation axis
        u_y = float(rot_axis[1])  # 2-component of the rotation axis
        u_z = float(rot_axis[2])  # 3-component of the rotation axis

        # compute the rotation matrix for the twist rotation
        rot_matrix_Tw = np.asarray([
        [cos(alpha_tw) + u_x**2*(1-cos(alpha_tw)),          u_x*u_y*(1-cos(alpha_tw)) - u_z*sin(alpha_tw),	u_x*u_z*(1-cos(alpha_tw)) + u_y*sin(alpha_tw)],
        [u_y*u_x*(1-cos(alpha_tw)) + u_z*sin(alpha_tw),	cos(alpha_tw) + u_y**2*(1-cos(alpha_tw)),          u_y*u_z*(1-cos(alpha_tw)) - u_x*sin(alpha_tw)],
        [u_z*u_x*(1-cos(alpha_tw)) - u_y*sin(alpha_tw),    u_z*u_y*(1-cos(alpha_tw)) + u_x*sin(alpha_tw),    cos(alpha_tw) + u_z**2*(1-cos(alpha_tw))]
        ]);

        # compute the new directors
        d1 = rot_matrix_Tw@ d1_noTw
        d2 = rot_matrix_Tw @ d2_noTw

        # transpose the vectors for output
#         d1 = d1';
#         d2 = d2';
#         d3 = d3';

        # write the computed current director and reference point to the output-array
        temp = np.expand_dims(arr_xre[i,:],0)
        temp = np.hstack([temp,d1.T,d2.T,d3.T])
        beam.arr_coordinates.append(temp[0,:])
        



    beam.arr_coordinates = np.asarray(beam.arr_coordinates)
    # beam connectivities
    for i in range(model["M_struc"]):
        beam.connectivity[i,:] = [i,i+1,i];


In [79]:
def if_assign(field, subkey, xq, sep_values=False):
    """
        This function checks if the subkey is available in the field.keys
        if it does the it does linear interpolation between grid and values
        Inputs:
            field: dict
            subkey: str
            xq: querry points
    """
    if not sep_values:
        if subkey in field.keys():
            x = field[subkey]["grid"]
            y = field[subkey]["values"]
            fun = interp1d(x,y)
            return fun(xq)
    else:
        if subkey in field.keys():
            x = field[subkey]["grid"]
            y = sep_values
            fun = interp1d(x,y)
            return fun(xq)
def interp1D(array, xq):
    x = array[:,0]
    y = array[:,1]
    fun = interp1d(x,y)
    return fun(xq)
class UVLM:
    def __init__(self, strName,model_uvlm,airfoils):
        # span-wise and chord-wise discretization
        self.M = model_uvlm["M_aero"]
        self.N = model_uvlm["N_aero"]
        
        # natural coordinates of span-and chord-wise discretization
        self.xhi_x  = np.arange(0,1 + 1/self.M,1/self.M) # span-wise
        self.xhi_y_c  = np.arange(0,1 + 1/self.N,1/self.N) # chord-wise
        
        nbr             = 0;                    # number of blade root cross-section
        arr_inz_airfoil = [];                   # index array for locating airfoils in span-wise direction
        
        # interpolating coordinates of reference axis according to span-wise discretization
        self.arr_xre_x = if_assign(model_uvlm["reference_axis"], "x", self.xhi_x)
        self.arr_xre_y = if_assign(model_uvlm["reference_axis"], "y", self.xhi_x)
        self.arr_xre_z = if_assign(model_uvlm["reference_axis"], "z", self.xhi_x)
        arr_val_c = []
        arr_val_w = []
        # if-statement according to type of surface cross-section
        if "blade" in strName:
            self.airfoil = model_uvlm["airfoil_position"]
            nbr              = model_uvlm["blade_root_position"]

            # interpolating chord length in span-wise direction
            self.arr_c = if_assign(model_uvlm, "chord", self.xhi_x)

            # interpolating twist angle according to span-wise discretization
            self.arr_twist = np.zeros((self.M,1))
            
            self.arr_twist = if_assign(model_uvlm, "twist", self.xhi_x)
            
            # interpolating airfoil in span-wise direction. This is important
            # to identify the blade root and blade's lifting surfaces
            airfoil_grid    = self.airfoil["grid"]
            # TODO:
#             airfoil_values  = [1:length(uvlm_obj.airfoil.grid)]
#             print(self.airfoil)
#             print('*'*80)
#             print(len(airfoils))
#             print(airfoils[0].keys())
            airfoil_values  = range(len(self.airfoil["grid"]))
            fun = interp1d(airfoil_grid,airfoil_values)
            arr_inz_airfoil = np.fix(fun(self.xhi_x))

            arr_pitch_ax = np.ones((self.M+1,1))*1.0;
            self.arr_pitch_ax = if_assign(model_uvlm, "pitch_axis",self.xhi_x)
            
            # natural coordinates of chord-wise discretization for whole
            # surface of cross-section
            xhi_y_w = self.xhi_y_c;
        elif "pipe" in strName:
            # interpolating chord length in span-wise direction
            self.arr_c          = if_assign(model_uvlm,"outer_diameter",
                                            self.xhi_x,model_uvlm["outer_diameter"]["values"]-model_uvlm["thickness"]["values"]
                                           );
            self.arr_twist      = np.zeros((self.M+1,1)) # zero twist
            self.arr_pitch_ax   = np.ones((uvlm_obj.M+1,1))*0.5; # location of pitch axis
            self.airfoil["labels"] = ['circular','circular']  # artificial "airfoil" sections for pipe
            self.airfoil["grid"]   = [0.0, 1.0];               # artificial "airfoil" sections grid for pipe

            # mesh discretization around circular cross-section
            temp = np.linspace(0,np.pi, self.N+1)
            xhi_y_w = 0.5*(1-np.cos(temp))
        # loop over airfoils in the model to calculate coordinates of
        # cross-section surfaces
        for i in range(len(self.airfoil["labels"])):
            airfoil_name = self.airfoil["labels"][i]

            # searching current airfoil from airfoil-list
            for j in range(len(airfoils)):
                if len(airfoils) == 1:
                    if airfoil_name in airfoils[j]["name"]:
                        airfoilj = airfoils;
                        break

                else:
                    if airfoil_name in airfoils[j]["name"]  :
                        airfoilj = airfoils[j]
                        break




            # extracting coordinates for upper and lower airfoil 
            airfoil_coord = np.vstack([airfoilj["coordinates"]["x"],airfoilj["coordinates"]["y"]]).T
#             print("airfoil_coord.shape: ",airfoil_coord.shape)
            inz0 = np.nonzero(airfoil_coord[:,0]==0) 
            inz0 = inz0[0][0] + 1
#             print("inz0:", inz0)
            airfoil_coord_u = airfoil_coord[:inz0,:]; 
            inzsort = np.argsort(airfoil_coord_u[:,0])
            airfoil_coord_u = airfoil_coord_u[inzsort,:]
            airfoil_coord_l = airfoil_coord[inz0-1:,:]; 
            inzsort = np.argsort(airfoil_coord_l[:,0]) 
            airfoil_coord_l = airfoil_coord_l[inzsort,:]

            # interpolating values in airfoil thickness direction according to
            # discretization in chord-wise direction to calculate camber
            # surface coordinates
#             print("airfoil_coord_u: ",airfoil_coord_u)
            xhi_airf_u = interp1D(airfoil_coord_u,self.xhi_y_c);
            xhi_airf_l = interp1D(airfoil_coord_l,self.xhi_y_c);
            # coordinates of camber surface
#             print(xhi_airf_u.shape, xhi_airf_l.shape)
            xhi_airf_c = (xhi_airf_u + xhi_airf_l)/2;

            # interpolating values in airfoil thickness direction according to
            # discretization in chord-wise direction to calculate whole
            # cross-section surface coordinates
            xhi_airf_u = interp1D(airfoil_coord_u,xhi_y_w);
            xhi_airf_l = interp1D(airfoil_coord_l,xhi_y_w);
            # coordinates of whole cross-section surface
            xhi_airf_w = np.append(np.append(xhi_airf_u[:-1],(xhi_airf_u[-1] + xhi_airf_l[-1])/2),xhi_airf_l[-2::-1])
#             print("xhi_airf_w.shape: ",xhi_airf_w.shape)
            
            # if-statement to detect, if blade root s. In case blade root
            # , switching from whole surface to camber surface
            if i == nbr+1:
                if nbr != 0:
                    xhi_airf_w = np.append(xhi_airf_c[:-1],xhi_airf_c[::-1])
#                     print("inside: xhi_airf_w.shape: ",xhi_airf_w.shape)



#             print("xhi_airf_c.shape :", xhi_airf_c.shape)
            arr_val_c.append( xhi_airf_c)      # camber surface coordinates 
            arr_val_w.append( xhi_airf_w) # whole surface coordinates
        arr_val_c = np.asarray(arr_val_c).T
        arr_val_w = np.asarray(arr_val_w).T
        

        # interpolating camber surface coordinates in span-wise direction 
        arr_xhi_airf_c = np.zeros((self.xhi_x.size, arr_val_c.shape[0]))
        for i in range(arr_val_c.shape[0]):
            x = np.asarray(self.airfoil["grid"])
            y = arr_val_c[i,:]
            # print("line 165: (x.shape, y.shape): ",x.shape, y.shape)
            fun = interp1d(x,y)
            arr_val = fun(self.xhi_x)
            arr_xhi_airf_c[:,i] = arr_val


        # interpolating whole surface coordinates in span-wise direction
        arr_xhi_airf_w =  np.zeros((self.xhi_x.size, arr_val_w.shape[0]))
        for i in range(arr_val_w.shape[0]):
            x = self.airfoil["grid"]
            y = arr_val_w[i,:]
            fun = interp1d(x,y)
            arr_val = fun(self.xhi_x)
            
            arr_xhi_airf_w[:,i] = arr_val


        if nbr!=0:
            inz  = np.nonzero(arr_inz_airfoil<=nbr)
            inz = inz[0]
            temp_coo = arr_xhi_airf_c[inz[-1]+1,:]
            arr_xhi_airf_w[inz[-1]+1,:] = np.append(temp_coo,temp_coo[-2::-1])
        self.arr_xhi_airf_c  = arr_xhi_airf_c;
        self.arr_xhi_airf_w  = arr_xhi_airf_w;
        self.xhi_y_w         = xhi_y_w;
        self.arr_xhi_x       = self.xhi_x
        self.nbr             = nbr;
        self.arr_inz_airfoil = arr_inz_airfoil;

            
            
    
    
        
def fun_extract_uvlm_data(strName,model_uvlm,airfoils):
    """   
     input:
       strName - type of surface of (airfoil) cross-section
       model - component uvlm object specified in WindIO
       airfoils - airfoil data specified in WindIO
     output:
         uvlm_ob - uvlm object containing coordinates and connectivity for creating aerodynamic grid
     =================================================================================================================
    """
    uvlm_ob = UVLM(strName,model_uvlm,airfoils)
    # for debugging
#     for var in vars(uvlm_ob):
#         print(f"{var} : ",getattr(uvlm_ob, var))
#         print('*'*80)
#         print('*'*80)
#         print('*'*80)
    return uvlm_ob
blade_Aero  = fun_get_uvlm_geometry('blade',
                                        data["components"]["blade"]["DeSiO"]["uvlm"],
                                        data["airfoils"],scaling_blade)

(51, 3)


In [80]:
def fun_get_uvlm_geometry(strName,model,airfoils,scale_opt=1):
    """
         Input:
           airfoils - airfoil data specified in WindIO
           scale_opt - scaling factor for scaling in longitudinal direction (optional input)
         output:
             uvlm_ob - uvlm object containing coordinates and connectivity for creating aerodynamic grid
         =================================================================================================================

    """
    # extracting uvlm data from WindIO
    uvlm_ob = fun_extract_uvlm_data(strName,model,airfoils)
    
    # elements in x (span) and y (chord) direction
    mx =  len(uvlm_ob.arr_xhi_x)-1; 
    my =  uvlm_ob.N;
    
    # natural coordinates in chord-wise direction of cross-section
    xhi_y_c   = uvlm_ob.xhi_y_c;                                            # camber surface
    xhi_y_w   = np.append(uvlm_ob.xhi_y_w, uvlm_ob.xhi_y_w[-2::-1])       # whole surface of cross-section
    
    # some extracted geometry arrays interpolated to spatial discretization
    arr_chord      = uvlm_ob.arr_c;                                         # chord length c
    arr_pitch_ax   = uvlm_ob.arr_pitch_ax;                                  # position of pitch axis in c
    arr_xhi_airf_c = uvlm_ob.arr_xhi_airf_c;                                # camber surface of (ariofoil) cross-section in c
    arr_xhi_airf_w = uvlm_ob.arr_xhi_airf_w;                                # whole surface of (ariofoil) cross-section in c
    arr_twist      = -uvlm_ob.arr_twist;                                    # twist angle around pitch axis
    arr_xre        = np.vstack([uvlm_ob.arr_xre_x,uvlm_ob.arr_xre_y, uvlm_ob.arr_xre_z*scale_opt]).T       # reference axis
    # until fun_get_uvlm_geometry matlab code Line 45 is done have to start from for loop
    return 
blade_Aero  = fun_get_uvlm_geometry('blade',
                                        data["components"]["blade"]["DeSiO"]["uvlm"],
                                        data["airfoils"],scaling_blade)

(51, 3)


# Blades

In [81]:
n_blades = 0;
if 'number_of_blades' in data["assembly"].keys():
    n_blades = data["assembly"]["number_of_blades"];
if "blade" in data["components"].keys() and flag_blade == 1:
#     blade_Aero  = fun_get_uvlm_geometry('blade',data["components"]["blade"]["DeSiO"]["uvlm"],data["airfoils"],scaling_blade);
    blade_Struc = fun_get_beam_model('blade',data["components"]["blade"]["DeSiO"]["beam"],data["materials"],scaling_blade)
    blade_Aero  = fun_get_uvlm_geometry('blade',
                                        data["components"]["blade"]["DeSiO"]["uvlm"],
                                        data["airfoils"],scaling_blade)
    

(51, 3)
